## Workshop om Ragas

## Begrepsforklaringer
##### RAG
RAG står for Retrieval-Augmented Generation og gir generativ kunstig intelligens modeler informasjonsinnhenting muligheter. Dette betyr RAG-en hjelper generative KI ved å gi den tilgang til relevant informasjon som kan hjelpe med å svare på spørsmål fra bruker. 
Informasjonen blir på forhånd lagret omgjort til LLM-embeddings (store vektorer) og lagret i vektor-databaser. Før den generative KI-en svarer, blir brukerspørsmålet gjort om til en egen LLM-embedding og den embeddingen blir brukt til å sammenligne med de andre vektorene i vektor-databasen. Her blir ofte "Cosine-similarity" brukt for å finne vektorene som ligner mest. Deretter blir informasjonen som er mest relevant til spørsmålet, gitt til KI-en.

### Hva er ragas?
Ragas er et bra dokumentert (!), open-source bibliotek som lar deg evaluere LLM-applikasjoner og RAG-er. Viktig for denne evalueringen er metrics.

<img src="metrics_mindmap.png" alt="Metrics Mindmap" width="500"/>

To typer metrics:

##### LLM Based
- Bruker LLM til å vurdere. 
- Non-deterministisk ved at LLM ikke alltid vil returnere det samme resultatet
- Likevel vist seg å være mer nøyaktige og nærmere menneskelig evaluering

##### Non-LLM Based
- Bruker **ikke** LLM til å vurdere
- Deterministiske 
- Bruker tradisjonelle metoder for å evaluere
- Mindre nøyaktige sammenlignet med menneskelig evaluering


To andre kategorier:
##### Single Turn Metrics
- Evaluerer basert på én runde med interaksjon mellom bruker og generativ KI

##### Multiple Turn Metrics
- Evalierer basert på flere runder med interaksjon mellom bruker og generativ KI


### Hvordan evaluere med Ragas?
For å evaluere hvor god en generativ KI trenger man 3 ting:
- Spørsmål
- Svar
- Referanse/riktig svar

Med dette kan man evaluere om svaret KI-en gir stemmer opp mot svaret vi forventer.

For å evaluere hvor god en RAG er til å gi riktig informasjon til KI-en trenger man 4 ting:
- Spørsmål
- Svar
- Referanse/riktig svar
- Gitt kontekst/informasjon

Den siste (gitt kontekst) er viktig når man skal vurdere hvor svikten i system ligger. Hvis det er gitt feil kontekst, er det RAG-en som har mislyktes. Hvis det er riktig kontekst, men feil svar, er det KI-en som har mislyktes. 

Under skal vi se hvordan vi kan evaluere en RAG.

In [ ]:
%pip install -U ragas langchain langchain_openai faiss-cpu pandas

ERROR: Could not find a version that satisfies the requirement ragas_metrics (from versions: none)

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for ragas_metrics


### Sette env-variabler

In [5]:
import os

os.environ["LANGCHAIN_API_KEY"] = "your_key"
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_PROJECT"] = "Ragas Tutorial"

os.environ["OPENAI_API_KEY"] = "your_key"

### Lese fil og lage vektor-database

In [29]:
from ragas import EvaluationDataset
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter, MarkdownTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain.retrievers import ParentDocumentRetriever, EnsembleRetriever, MultiVectorRetriever
from langchain_community.retrievers import BM25Retriever
from langchain.storage import InMemoryStore, InMemoryByteStore
from langchain_core.documents import Document
from langchain_core.stores import BaseStore
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, ContextEntityRecall, ContextPrecision, FactualCorrectness
from langchain_community.vectorstores import FAISS
from ragas.utils import safe_nanmean
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
import pandas as ps

# Setter opp RAG-en

# Leser dokumentene vi vil legge inn i RAG-en
filepath = './dnd_doc.md'


dnd_document: str = ""
with open(filepath, encoding="utf-8") as f:
    dnd_document = f.read()

# Gjør teksten om til en Document-klasse fra Langchain
dnd_document = [Document(dnd_document)]

# Splitter dokumentet med en tekstsplitter fra Langchain
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0) # TextSplitter
splits = text_splitter.split_documents(dnd_document)

# Lager en vektor-database retriever
vector_retriever = FAISS.from_documents(splits, OpenAIEmbeddings()).as_retriever(
            search_type="similarity_score_threshold",
            search_kwargs = {
                "k": 2,
                "score_threshold": 0.7
                }
        )

### Generere svar til spørsmålene i CSV-filen

In [ ]:
llm = ChatOpenAI(
            model= "gpt-4o-mini",
            temperature=0,
            max_tokens=16384
        )

def convert_docs_to_strings(docs: list[Document]) -> list[str]:
        """Convert Objects of Document type to an list of strings"""
        return [doc.page_content for doc in docs]

def get_relevant_docs(query: str) -> list[str]:
    return convert_docs_to_strings(vector_retriever.invoke(input=query))

def generate_answer(query: str, relevant_doc: list[Document]):
        """Generate an answer for a given query based on the most relevant document."""
        prompt = f"question: {query}\n\nDocuments: {relevant_doc}"
        messages = [
            ("system", "You are a helpful assistant that answers questions based on given documents only."),
            ("human", prompt),
        ]
        ai_msg = llm.invoke(messages)
        return ai_msg.content

dataset = []

df = ps.read_csv("./sample_questions_dnd.csv")
querys = df["question"].tolist()
responses = df["answer"].tolist()

token_use = 0

for query, reference in zip(querys, responses):

    relevant_docs = get_relevant_docs(query=query)
    response = generate_answer(query, relevant_docs)
    dataset.append(
        {
            "user_input":query,
            "retrieved_contexts":relevant_docs,
            "response":response,
            "reference":reference
        }
    )

In [30]:
# Evaluerer datasettet

evaluation_dataset = EvaluationDataset.from_list(dataset)

evaluator_llm = LangchainLLMWrapper(llm)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), ContextPrecision(), FactualCorrectness()],
    llm=evaluator_llm
)

print(result)

Evaluating:   0%|          | 0/108 [00:00<?, ?it/s]

{'context_recall': 0.8333, 'context_precision': 0.8056, 'factual_correctness': 0.5525}
